<a href="https://colab.research.google.com/github/soares-f/JMeSH/blob/main/Approach_with_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extracting LitCovid in local env




In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/litcovid2pubtator.xml.gz

--2021-01-20 15:17:34--  https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/litcovid2pubtator.xml.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 2607:f220:41e:250::7, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 528283977 (504M) [application/x-gzip]
Saving to: ‘litcovid2pubtator.xml.gz’

litcovid2pubtator.x 100%[===================>] 503.81M  43.9MB/s    in 12s     

2021-01-20 15:17:46 (42.2 MB/s) - ‘litcovid2pubtator.xml.gz’ saved [528283977/528283977]



In [ ]:
!gunzip litcovid2pubtator.xml.gz

In [ ]:
!cat litcovid2pubtator.xml | grep C063533

<document><id>7347494</id><passage><infon key="name_3">surname:Dasa;given-names:Osama</infon><infon key="name_2">surname:Ahmed;given-names:Mustafa M.</infon><infon key="name_1">surname:Anderson;given-names:R. David</infon><infon key="article-id_pmid">32653420</infon><infon key="name_0">surname:Massoomi;given-names:Michael R.</infon><infon key="year">2020</infon><infon key="article-id_pmc">7347494</infon><infon key="article-id_publisher-id">S0002-9343(20)30550-7</infon><infon key="article-id_doi">10.1016/j.amjmed.2020.06.005</infon><infon key="title">Keywords</infon><infon key="type">front</infon><infon key="license">Since January 2020 Elsevier has created a COVID-19 resource centre with free information in English and Mandarin on the novel coronavirus COVID-19. The COVID-19 resource centre is hosted on Elsevier Connect, the company's public news and information website. Elsevier hereby grants permission to make all its COVID-19-related research that is available on the COVID-19 resourc

In [ ]:
!wc -l /content/drive/MyDrive/Research/BLAH7/sentences.txt

2011845 /content/drive/MyDrive/Research/BLAH7/sentences.txt


In [ ]:
pip install bioc

     |████████████████████████████████| 552kB 11.2MB/s 
     |████████████████████████████████| 5.8MB 27.6MB/s 
  Created wheel for bioc: filename=bioc-1.3.6-cp36-none-any.whl size=20269 sha256=36df6186e1d844ef717bca0012a8570bc079a13e78cc9164430474e93eb6824e
  Stored in directory: /root/.cache/pip/wheels/3f/91/3a/dca9e2d2f1deafa2c88ff05817448711d9262049e238813b9f
Successfully built bioc
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
filename = 'litcovid2pubtator.xml'

In [ ]:
from nltk import tokenize
import nltk
nltk.download('punkt')
import collections
import uuid 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def get_lens_passage(passage):
  sentences = tokenize.sent_tokenize(passage.text)
  lens_sentences = []
  ids_sentences = []
  for sentence in sentences:
    lens_sentences.append(len(sentence))
    ids_sentences.append(uuid.uuid4())
  return(lens_sentences,sentences, ids_sentences)

In [ ]:
def extract_annotations_passage(passage, type_of_annotation='MESH'):
  annotations_processed = []
  for annotation in passage.annotations:
    tmp =  {k.lower(): v for k, v in annotation.infons.items()}
    if type_of_annotation in tmp['identifier']:
      identifier = tmp['identifier']
      type_of_identifier = tmp['type']
      locations = []
      for location in annotation.locations:
        locations.append(location.offset)
      text = annotation.text
      annotations_processed.append({'identifier': identifier,
                                    'type_of_identifier': type_of_identifier,
                                    'locations': locations,
                                    'text': text})
  if len(annotations_processed) > 0:
    return(annotations_processed)
  else:
    return None

In [ ]:
def match_annotations_sentences(passage, file_handler=None, dict_annotations=None):
  annotations_processed = extract_annotations_passage(passage)
  lens_sentences, sentences,ids_sentences = get_lens_passage(passage)
  if file_handler is not None:
    for i in range(len(sentences)):
      file_handler.write(str(ids_sentences[i]) + '\t' + sentences[i] + '\n')
  else:
    for i in range(len(sentences)):
      print(str(ids_sentences[i]) + '\t' + sentences[i])
  #dict_annotations = collections.defaultdict(set)
  offsets_sentences = []
  offset_passage = passage.offset
  for i in range(len(lens_sentences)):
    if i == 0:
      offsets_sentences.append(lens_sentences[i]+passage.offset) 
    else:
      offsets_sentences.append(lens_sentences[i]+offsets_sentences[i-1])
  if annotations_processed is not None:
    for annotation in annotations_processed:
      min_location = min(annotation['locations'])
      #print(min_location)
      location_annotation = next((x[0] for x in enumerate(offsets_sentences) if x[1] >= min_location), len(offsets_sentences)-1)
      dict_annotations[annotation['identifier']].add(str(ids_sentences[location_annotation]))
  return(dict_annotations)

In [ ]:
import bioc
import io
# read from a ByteIO
dict_all = collections.defaultdict(set)
f_out = output = io.StringIO()
with open(filename, 'rb') as fp:
    reader = bioc.BioCXMLDocumentReader(fp)
    collection_info = reader.get_collection_info()
    i = 0
    for document in reader:
        for passage in document.passages:
          dict_all = match_annotations_sentences(passage, file_handler=f_out, dict_annotations=dict_all)
#          if tmp is not None:
#            dict_all = mergeDict(dict_all,tmp)
        i+=1
        print(i)
        if i % 30000==0:
          break


In [ ]:
import pickle
with open('/content/drive/MyDrive/Research/BLAH7/filename.pickle', 'wb') as handle:
    pickle.dump(dict_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

import shutil

with open('/content/drive/MyDrive/Research/BLAH7/sentences.txt', 'w') as fd:
  f_out.seek(0)
  shutil.copyfileobj(f_out, fd, -1)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Research/BLAH7/filename.pickle', 'rb') as handle:
  dict_all = pickle.load(handle)

In [ ]:
tmp_passage1 = extract_annotations_passage(document.passages[1])

In [ ]:
with open('/content/drive/MyDrive/Research/BLAH7/occurences_v1.tsv','w') as f:
  for key in dict_all:
    f.write(key + '\t' +  str(len(dict_all[key])) + '\n')

In [ ]:
with open('/content/drive/MyDrive/Research/BLAH7/initial_terms_to_translate.txt','r') as f:
  terms_to_translate = f.readlines()

In [ ]:
del(f_out)